# 1. Package 설치

In [ ]:
%pip install --upgrade pymilvus langchain openai tiktoken

# 2. Knowledge Base 구성을 위한 데이터 생성

- 4. LangChain을 활용한 Vector Database 변경 (Chroma -> Milvus)과 동일함

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=128,
)

# 개인정보 보호법
document_name = './privacy_with_markdown.docx'

loader = Docx2txtLoader(document_name)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
len_list = len(document_list)
len_list

86

In [3]:
document_list[len_list - 10]

Document(metadata={'source': './privacy_with_markdown.docx'}, page_content='1. 제25조제5항(제26조제8항에 따라 준용되는 경우를 포함한다)을 위반하여 고정형 영상정보처리기기의 설치 목적과 다른 목적으로 고정형 영상정보처리기기를 임의로 조작하거나 다른 곳을 비추는 자 또는 녹음기능을 사용한 자\n\n2. 제59조제1호를 위반하여 거짓이나 그 밖의 부정한 수단이나 방법으로 개인정보를 취득하거나 개인정보 처리에 관한 동의를 받는 행위를 한 자 및 그 사정을 알면서도 영리 또는 부정한 목적으로 개인정보를 제공받은 자\n\n3. 제60조를 위반하여 직무상 알게 된 비밀을 누설하거나 직무상 목적 외에 이용한 자\n\n\n\n제73조(벌칙) ① 다음 각 호의 어느 하나에 해당하는 자는 2년 이하의 징역 또는 2천만원 이하의 벌금에 처한다.\n\n1. 제36조제2항(제26조제8항에 따라 준용되는 경우를 포함한다)을 위반하여 정정ㆍ삭제 등 필요한 조치를 하지 아니하고 개인정보를 계속 이용하거나 이를 제3자에게 제공한 자\n\n2. 제37조제2항(제26조제8항에 따라 준용되는 경우를 포함한다)을 위반하여 개인정보의 처리를 정지하지 아니하고 개인정보를 계속 이용하거나 제3자에게 제공한 자\n\n3. 국내외에서 정당한 이유 없이 제39조의4에 따른 비밀유지명령을 위반한 자\n\n4. 제63조제1항(제26조제8항에 따라 준용되는 경우를 포함한다)에 따른 자료제출 요구에 대하여 법 위반사항을 은폐 또는 축소할 목적으로 자료제출을 거부하거나 거짓의 자료를 제출한 자\n\n5. 제63조제2항(제26조제8항에 따라 준용되는 경우를 포함한다)에 따른 출입ㆍ검사 시 자료의 은닉ㆍ폐기, 접근 거부 또는 위조ㆍ변조 등을 통하여 조사를 거부ㆍ방해 또는 기피한 자\n\n② 제1항제3호의 죄는 비밀유지명령을 신청한 자의 고소가 없으면 공소를 제기할 수 없다.\n\n[전문개정 2023. 3. 14.]')

In [4]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

True

In [5]:
import os

from langchain.vectorstores.zilliz import Zilliz

# 1. Set up the name of the collection to be created.
COLLECTION_NAME = 'privacy_law_index'

# 2. Set up the dimension of the embeddings.
DIMENSION = 768

# 3. Set up the cohere api key
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# 4. Set up the connection parameters for your Zilliz Cloud cluster.
URI = os.environ["MILVUS_CLUSTER_ENDPOINT"]

# 5. Set up the token for your Zilliz Cloud cluster.
# You can either use an API key or a set of cluster username and password joined by a colon.
TOKEN = os.environ["MILVUS_TOKEN"]

In [6]:
# Connect using a MilvusClient object
from pymilvus import MilvusClient, DataType

# 1. Set up a Milvus client
client = MilvusClient(
    uri=URI,
    token=TOKEN
)

res = client.get_load_state(
    collection_name=COLLECTION_NAME
)
print(f"{COLLECTION_NAME} state: {res['state']}")
res_state = str(res['state'])

privacy_law_index state: Loaded


In [7]:
is_create = False
if res_state == 'NotExist':
    is_create = True
print(f"is_create: {is_create}")

if is_create:
    embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
    connection_args = { 'uri': URI, 'token': TOKEN }

    ids = [str(i) for i in range(len(document_list))]

    vector_store = Zilliz(
        embedding_function=embeddings, 
        connection_args=connection_args,
        collection_name=COLLECTION_NAME,
        drop_old=True,
        auto_id=True
    ).from_documents(
        document_list,
        embedding=embeddings,
        collection_name=COLLECTION_NAME,
        connection_args=connection_args,
        ids = ids
    )

is_create: False


In [41]:
query = '개인정보 유출 시 대응 방법은 무엇인가요?'

# 3. 답변 생성을 위한 Retrieval

- RetrievalQA에 전달하기 위해 retriever 생성
- search_kwargs의 k값을 변경해서 가져올 문서의 갯수를 지정할 수 있음
- .invoke()를 호출해서 어떤 문서를 가져오는지 확인 가능

In [42]:
retriever = vector_store.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(metadata={'source': './privacy_with_markdown.docx', 'pk': '41'}, page_content='⑪ 공공기관 외의 개인정보처리자는 개인정보파일 운용으로 인하여 정보주체의 개인정보 침해가 우려되는 경우에는 영향평가를 하기 위하여 적극 노력하여야 한다.<개정 2023. 3. 14.>\n\n\n\n제34조(개인정보 유출 등의 통지ㆍ신고) ① 개인정보처리자는 개인정보가 분실ㆍ도난ㆍ유출(이하 이 조에서 “유출등”이라 한다)되었음을 알게 되었을 때에는 지체 없이 해당 정보주체에게 다음 각 호의 사항을 알려야 한다. 다만, 정보주체의 연락처를 알 수 없는 경우 등 정당한 사유가 있는 경우에는 대통령령으로 정하는 바에 따라 통지를 갈음하는 조치를 취할 수 있다. <개정 2023. 3. 14.>\n\n1. 유출등이 된 개인정보의 항목\n\n2. 유출등이 된 시점과 그 경위\n\n3. 유출등으로 인하여 발생할 수 있는 피해를 최소화하기 위하여 정보주체가 할 수 있는 방법 등에 관한 정보\n\n4. 개인정보처리자의 대응조치 및 피해 구제절차\n\n5. 정보주체에게 피해가 발생한 경우 신고 등을 접수할 수 있는 담당부서 및 연락처\n\n② 개인정보처리자는 개인정보가 유출등이 된 경우 그 피해를 최소화하기 위한 대책을 마련하고 필요한 조치를 하여야 한다.<개정 2023. 3. 14.>\n\n③ 개인정보처리자는 개인정보의 유출등이 있음을 알게 되었을 때에는 개인정보의 유형, 유출등의 경로 및 규모 등을 고려하여 대통령령으로 정하는 바에 따라 제1항 각 호의 사항을 지체 없이 보호위원회 또는 대통령령으로 정하는 전문기관에 신고하여야 한다. 이 경우 보호위원회 또는 대통령령으로 정하는 전문기관은 피해 확산방지, 피해 복구 등을 위한 기술을 지원할 수 있다.<개정 2013. 3. 23., 2014. 11. 19., 2017. 7. 26., 2020. 2. 4., 2023. 3. 14.>\n\n④ 제1항에 따른 유출등의 통지 및 

# 4. Augmentation을 위한 Prompt 활용

- Retrieval 된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용

In [23]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

# 5. 답변 생성

- RetrievalQA를 통해 LLM에 전달
    - RetrievalQA는 create_retrieval_chain으로 대체됨
    - 실제 ChatBot 구현 시 create_retrieval_chain으로 변경하는 과정을 볼 수 있음
- 하단의 dictionary_chain과 연계하여 사용

In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [25]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [26]:
ai_message = qa_chain.invoke({"query": query})

In [27]:
ai_message

{'query': '개인정보 유출 시 대응 방법은 무엇인가요?',
 'result': '개인정보 유출 시, 개인정보처리자는 지체 없이 정보주체에게 유출 사실과 관련된 사항을 통지해야 합니다. 또한, 피해를 최소화하기 위한 대책을 마련하고 필요한 조치를 취해야 하며, 대통령령으로 정하는 바에 따라 보호위원회나 전문기관에 신고해야 합니다. 정보주체는 손해가 발생한 경우 손해배상을 청구할 수 있습니다.'}

# 6. Retrieval을 위한 keyword 사전 활용

- Knowledge Base에서 사용되는 keyword를 활용하여 사용자 질문 수정
- LangChain Expression Language (LCEL)을 활용한 Chain 연계

In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 개인정보 보호법
dictionary = [
    "개인정보 보호법 위반을 나타내는 표현 -> 개인정보 보호법 위반",
    "개인정보 주체의 권리를 나타내는 표현 -> 개인정보 주체 권리",
    "개인정보 유출을 나타내는 표현 -> 개인정보 유출",
    "불법 수집을 나타내는 표현 -> 불법 수집",
    "정보 보안을 나타내는 표현 -> 정보 보안"
]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요.
    사전: {dictionary}

    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [29]:
new_question = dictionary_chain.invoke({"question": query})

In [30]:
new_question

'개인정보 유출 시 대응 방법은 무엇인가요?'

In [31]:
law_chain = {"query": dictionary_chain} | qa_chain

In [32]:
ai_response = law_chain.invoke({"question": query})

In [33]:
ai_response

{'query': '개인정보 유출 시 대응 방법은 무엇인가요?',
 'result': '개인정보가 유출되었을 때, 개인정보처리자는 지체 없이 해당 정보주체에게 유출된 개인정보의 항목, 시점 및 경위, 피해 최소화 방법, 대응 조치 및 피해 구제 절차, 신고 접수 담당부서 및 연락처를 알려야 합니다. 또한, 피해를 최소화하기 위한 대책을 마련하고 필요한 조치를 취해야 합니다. 개인정보의 유출 경로와 규모 등을 고려하여 보호위원회나 전문기관에 신고해야 합니다.'}